<a href="https://colab.research.google.com/github/arjunmenonv/RLGames/blob/master/Minesweeper/Minesweeper_Reinforce_with_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!cp "" 

cp: missing destination file operand after 'gdrive/My Drive/Colab Notebooks/RL_Games_Minesweeper/minesweeper_pygame.py'
Try 'cp --help' for more information.


In [ ]:
!pip install pygame
!pip install mss

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import gym
import collections
import numpy as np

from tensorflow.keras import optimizers
tf.compat.v1.disable_eager_execution()
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.initializers import glorot_normal
from tensorflow.keras.optimizers import Adam

import warnings
warnings.filterwarnings("ignore")


In [ ]:
from minesweeper_pygame import Minesweeper
env = Minesweeper(display=False, ROWS = 6, COLS = 6, MINES = 6,OUT='CONDENSED', rewards = {"win" : 1, "loss" : -1, "progress" : 0.9, "noprogress" : -0.3, "YOLO" : -0.3})
min_state = env.stateConverter(env.get_state()).flatten()

In [ ]:
RANDOM_SEED=1
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [ ]:
def get_winrate(env,actor):
    games = 0
    moves = 0
    stuck = 0
    won_games = 0
    lost_games = 0
    r = 0
    while games < 10000:
        while True:
            state = env.stateConverter(env.get_state())

            action_probs = actor.predict(np.array([state]))
            action_probs=action_probs.ravel()

            action = np.random.choice(np.arange(len(action_probs)), p=action_probs)
            next_state, reward, done, _ = env.step(action)

            moves += 1

            if reward == 1:
                won_games += 1
            if reward == -1:
                lost_games += 1

            if done:
                games += 1
                env.reset()
                moves = 0
                break
            elif moves >= 30:
                stuck += 1
                games += 1
                
                lost_games +=1
                env.reset()
                moves = 0
                break
    return(won_games/games)


In [ ]:
#Note: The advantage here is the total rewards. Hence, y_true during train_on_batch has magnitude of the total rewards!
# I have added an extra 1 with y_pred so that the loss does not go to infinity and to avoid NaN error
def custom_loss(y_true,y_pred):
        loss =  -tf.reduce_mean(y_true*tf.math.log(y_pred+1))
        return loss


In [ ]:
def PolicyEstimator(env,lr=0.0001):
  # Neural network
  n_inputs=env.stateConverter(env.get_state()).shape
  n_hidden = 288
  n_hidden2 = 220
  n_hidden3 = 200
  n_outputs = env.ROWS*env.COLS

  with strategy.scope():


    model = Sequential()

    model.add(Conv2D(18, kernel_size=(5, 5), strides=(1, 1),kernel_initializer=glorot_normal(), activation='relu',padding='same', input_shape=n_inputs))
    model.add(Conv2D(36, kernel_size=(3, 3), strides=(1, 1),kernel_initializer=glorot_normal(),activation='relu', padding='same'))
    model.add(Flatten())
    model.add(Dense(n_hidden, activation='relu'))
    model.add(Dense(n_hidden2, activation='relu'))
    model.add(Dense(n_hidden3, activation='relu'))

    model.add(Dense(n_outputs, activation='softmax'))

    model.compile(loss=custom_loss, optimizer=Adam(learning_rate=lr))

  print(model.summary())

  return model


In [ ]:
def ValueEstimator(env,lr=0.0001):
  # Neural network
  n_inputs=env.stateConverter(env.get_state()).shape
  n_hidden = 288
  n_hidden2 = 220
  n_hidden3 = 200


  with strategy.scope():


    model = Sequential()

    model.add(Conv2D(18, kernel_size=(5, 5), strides=(1, 1),kernel_initializer=glorot_normal(), activation='relu',padding='same', input_shape=n_inputs))
    model.add(Conv2D(36, kernel_size=(3, 3), strides=(1, 1),kernel_initializer=glorot_normal(),activation='relu', padding='same'))
    model.add(Flatten())
    model.add(Dense(n_hidden, activation='relu'))
    model.add(Dense(n_hidden2, activation='relu'))
    model.add(Dense(n_hidden3, activation='relu'))

    model.add(Dense(1, activation=None))

    model.compile(loss='mse', optimizer=Adam(learning_rate=lr))

  print(model.summary())

  return model


In [ ]:
def reinforce_with_baseline(env,lr=0.0001,num_episodes=5000,discount_factor=0.99):

    actor = PolicyEstimator(env,lr)
    baseline=ValueEstimator(env,lr)

    wins=0

    
    Transition = collections.namedtuple("Transition", ["state", "action", "reward", "next_state", "done"])

    episode_lengths=np.zeros(num_episodes)
    episode_rewards=np.zeros(num_episodes)



    for game in range(num_episodes):

        done = False
        env.reset()
        state= env.stateConverter(env.get_state())

        episode = []
        t_step=0

        states, actions, rewards = [], [], []
        while not done and t_step<6*6:
            t_step += 1


            action_probs = actor.predict(np.array([state]))
            action_probs=action_probs.ravel()

            # print(action_probs)


            if np.isnan(action_probs).any():
              break


            # for i in range(len(action_probs)):
            #   if i not in env.action_space():
            #     action_probs[i]=0
              
            # action_probs/=sum(action_probs)

            


            action = np.random.choice(np.arange(len(action_probs)), p=action_probs)
            next_state, reward, done, _ = env.step(action)

            if reward==1:
              wins+=1


            # next_state=next_state.flatten()
            episode.append(Transition(
              state=state, action=action, reward=reward, next_state=next_state, done=done))
            state = next_state




        flag=0
        for t, transition in enumerate(episode):

            # if not done:
            #   break


            # The return after this timestep
            total_return = sum(discount_factor**i * trans.reward for i, trans in enumerate(episode[t:]))
            if flag==0:

              recorded_return=total_return
              flag=1

            # Calculate baseline/advantage
            baseline_value = baseline.predict(np.array([transition.state]))


            advantage = total_return- baseline_value

            one_hot_output = np.zeros(36)
            one_hot_output[transition.action] = advantage + 1e-5

            baseline.train_on_batch(np.array([transition.state]),np.array([total_return]))

            actor.train_on_batch( np.array([transition.state]), np.expand_dims(one_hot_output,axis=0))

            


        print('episode:',(game+1),'score:',(recorded_return),'timesteps:',(t_step))

        episode_rewards[game] += recorded_return
        episode_lengths[game] = t_step


    wins=get_winrate(env,actor)
    print('win_rate=')
    print(wins*100)

    actor.save('/content/gdrive/My Drive/Colab Notebooks/RL_Games_Minesweeper/Models/reinforce_with_baseline_model_v2.h5')


    plt.figure(figsize=(10,6))
    plt.subplot(211)
    plt.plot(episode_rewards)
    plt.xlabel('episodes'); plt.ylabel('rewards')
    plt.xlim((0, len(episode_rewards)))
    plt.legend(loc=1); plt.grid()
    plt.subplot(212)
    plt.plot(episode_lengths)
    plt.xlabel('episodes'); plt.ylabel('time_steps')
    plt.xlim((0, len(episode_lengths)))
    plt.legend(loc=4); plt.grid()
    plt.tight_layout(); plt.show()

    return actor
         


In [ ]:
%time
agent = reinforce_with_baseline(env,num_episodes=200000)

In [ ]:
print('hello')

hello
